# Week 12 — j-Stability Across Regimes
We simulate multiple environments with a stable causal core and a few unstable edges.
The stable edges should appear consistently across regimes.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

rng = np.random.default_rng(3)
d = 5
A_base = np.array([
    [0,1,0,0,0],
    [0,0,1,0,0],
    [0,0,0,1,0],
    [0,0,0,0,1],
    [0,0,0,0,0],
], dtype=int)

def topo_order(A):
    d = A.shape[0]
    indeg = A.sum(0).astype(int).tolist()
    order, S = [], [i for i in range(d) if indeg[i]==0]
    while S:
        v = S.pop()
        order.append(v)
        for w in range(d):
            if A[v,w] != 0:
                indeg[w] -= 1
                if indeg[w] == 0:
                    S.append(w)
    return order

def sample_sem(W, n, rng):
    d = W.shape[0]
    order = topo_order((W != 0).astype(int))
    X = np.zeros((n, d), dtype=float)
    for t in range(n):
        for j in order:
            parents = np.where(W[:, j] != 0)[0]
            mean = float(np.dot(X[t, parents], W[parents, j])) if len(parents) else 0.0
            X[t, j] = mean + rng.normal(0, 1.0)
    return X

def add_extra_edges(A, k, rng):
    A = A.copy()
    order = topo_order(A)
    pos = {v:i for i,v in enumerate(order)}
    tries = 0
    while k > 0 and tries < 1000:
        tries += 1
        i = rng.integers(0, d-1)
        j = rng.integers(i+1, d)
        u, v = order[i], order[j]
        if A[u, v] == 0:
            A[u, v] = 1
            k -= 1
    return A

envs = ['e0','e1','e2']
datasets = []
edge_freq = np.zeros((d, d), dtype=float)

for env in envs:
    A_env = add_extra_edges(A_base, k=1, rng=rng)
    W = A_env * rng.uniform(0.6, 1.0, size=A_env.shape)
    X = sample_sem(W, 800, rng)
    df = pd.DataFrame(X, columns=[f'X{i}' for i in range(d)])
    datasets.append((env, df))

    # simple edge score by regression magnitude
    scores = np.zeros_like(A_env, dtype=float)
    for i in range(d):
        for j in range(d):
            if i == j:
                continue
            beta, *_ = np.linalg.lstsq(df[[f'X{i}']].values, df[f'X{j}'].values, rcond=None)
            scores[i, j] = abs(beta[0])
    edge_freq += (scores >= 0.25).astype(float)

edge_freq /= len(envs)

plt.figure(figsize=(5,3))
plt.imshow(edge_freq, cmap='viridis')
plt.colorbar(label='edge frequency across regimes')
plt.title('j-stability: edges stable across envs')
plt.xticks(range(d), [f'X{i}' for i in range(d)])
plt.yticks(range(d), [f'X{i}' for i in range(d)])
plt.tight_layout()
plt.show()
